<a href="https://colab.research.google.com/github/jianfeiZhao/Supply-Chain-Analysis/blob/master/pred_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

使用神经网络进行预测

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, f1_score

In [3]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/SupplyChain_clean.csv')
data.drop(['Unnamed: 0'], axis=1, inplace=True)
data.head(3)

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Category Id,Category Name,Customer City,Customer Country,Customer Segment,Customer State,Department Id,Department Name,Market,Order City,Order Country,Order Customer Id,Order Item Discount,Order Item Discount Rate,Order Item Product Price,Order Item Profit Ratio,Order Item Quantity,Order Region,Order State,Product Name,shipping date (DateOrders),Shipping Mode,Customer Fullname,order_year,order_month,order_weekday,order_hour,order_month_year,fraud,late_delivery
0,1,3,4,91.250000,314.640015,73,40,66,1,0,36,2,4,3,331,70,20755,13.110000,0.04,327.75,0.29,1,15,475,78,27149,3,1875,2018,1,2,22,36,0,0
1,3,5,4,-249.089996,311.359985,73,40,66,1,0,36,2,4,3,391,69,19492,16.389999,0.05,327.75,-0.80,1,13,841,78,2209,3,5374,2018,1,5,12,36,0,1
2,0,4,4,-247.779999,309.720001,73,40,452,0,0,5,2,4,3,391,69,19491,18.030001,0.06,327.75,-0.80,1,13,841,78,1980,3,4426,2018,1,5,12,36,0,0


In [4]:
# 数据集构造切分
X_data = data.loc[:, data.columns != 'fraud']
y_data = data['fraud']
X_fraud_train, X_fraud_test, y_fraud_train, y_fraud_test = train_test_split(X_data, y_data, test_size=0.2)

X_late = data.loc[:, data.columns != 'late_delivery']
y_late = data['late_delivery']
X_late_train, X_late_test, y_late_train, y_late_test = train_test_split(X_late, y_late, test_size=0.2)

# 数据规范化
sc = StandardScaler()
X_fraud_train = sc.fit_transform(X_fraud_train)
X_fraud_test = sc.transform(X_fraud_test)
X_late_train = sc.transform(X_late_train)
X_late_test = sc.transform(X_late_test)

In [5]:
# 使用神经网络进行分类
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

# BN层，在每个batch上将前一层的激活值重新规范化，使输出的数据呈0-1正态分布
keras.layers.BatchNormalization()
classifier = Sequential()
# 搭建神经网络
classifier.add(Dense(1024, activation='relu', input_dim=X_fraud_train.shape[1]))
classifier.add(Dense(512, activation='relu'))
classifier.add(Dense(256, activation='relu'))
classifier.add(Dense(128, activation='relu'))
classifier.add(Dense(64, activation='relu'))
classifier.add(Dense(32, activation='relu'))
classifier.add(Dense(1, activation='sigmoid'))
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

fraud 预测

In [7]:
# 模型训练
classifier.fit(X_fraud_train, y_fraud_train, batch_size=1024, epochs=10)

Epoch 1/10
142/142 [==============================] - 1s 4ms/step - loss: 0.0804 - accuracy: 0.9769
Epoch 2/10
142/142 [==============================] - 0s 3ms/step - loss: 0.0536 - accuracy: 0.9780
Epoch 3/10
142/142 [==============================] - 0s 3ms/step - loss: 0.0511 - accuracy: 0.9794
Epoch 4/10
142/142 [==============================] - 0s 3ms/step - loss: 0.0495 - accuracy: 0.9796
Epoch 5/10
142/142 [==============================] - 1s 4ms/step - loss: 0.0471 - accuracy: 0.9807
Epoch 6/10
142/142 [==============================] - 1s 4ms/step - loss: 0.0440 - accuracy: 0.9819
Epoch 7/10
142/142 [==============================] - 0s 3ms/step - loss: 0.0442 - accuracy: 0.9827
Epoch 8/10
142/142 [==============================] - 0s 3ms/step - loss: 0.0392 - accuracy: 0.9843
Epoch 9/10
142/142 [==============================] - 0s 3ms/step - loss: 0.0330 - accuracy: 0.9866
Epoch 10/10
142/142 [==============================] - 0s 3ms/step - loss: 0.0282 - accuracy: 0.9888

In [8]:
train_eval = classifier.evaluate(X_fraud_train,y_fraud_train)
test_eval = classifier.evaluate(X_fraud_test,y_fraud_test)
print('Train Evaluation:', train_eval)
print('Test Evaluation:', test_eval)

1129/1129 [==============================] - 2s 2ms/step - loss: 0.0400 - accuracy: 0.9856
Train Evaluation: [0.021225301548838615, 0.9920229911804199]
Test Evaluation: [0.03996586054563522, 0.9855971932411194]


In [12]:
y_fraud_pred = classifier.predict(X_fraud_test, batch_size=512, verbose=1)
y_fraud_pred = np.argmax(y_fraud_pred, axis=1)
print(accuracy_score(y_fraud_test, y_fraud_pred))

71/71 [==============================] - 0s 1ms/step
0.9792543762463993


late delivery 预测

In [6]:
# 模型训练
classifier.fit(X_late_train, y_late_train, batch_size=1024, epochs=10)

Epoch 1/10
142/142 [==============================] - 1s 5ms/step - loss: 0.0924 - accuracy: 0.9695
Epoch 2/10
142/142 [==============================] - 1s 4ms/step - loss: 0.0435 - accuracy: 0.9879
Epoch 3/10
142/142 [==============================] - 1s 4ms/step - loss: 0.0410 - accuracy: 0.9880
Epoch 4/10
142/142 [==============================] - 1s 4ms/step - loss: 0.0388 - accuracy: 0.9884
Epoch 5/10
142/142 [==============================] - 1s 4ms/step - loss: 0.0377 - accuracy: 0.9885
Epoch 6/10
142/142 [==============================] - 1s 4ms/step - loss: 0.0358 - accuracy: 0.9885
Epoch 7/10
142/142 [==============================] - 1s 4ms/step - loss: 0.0351 - accuracy: 0.9887
Epoch 8/10
142/142 [==============================] - 1s 4ms/step - loss: 0.0354 - accuracy: 0.9883
Epoch 9/10
142/142 [==============================] - 1s 4ms/step - loss: 0.0311 - accuracy: 0.9891
Epoch 10/10
142/142 [==============================] - 1s 4ms/step - loss: 0.0280 - accuracy: 0.9901

In [7]:
train_eval = classifier.evaluate(X_late_train,y_late_train)
test_eval = classifier.evaluate(X_late_test,y_late_test)
print('Train Evaluation:', train_eval)
print('Test Evaluation:', test_eval)

1129/1129 [==============================] - 2s 2ms/step - loss: 0.0461 - accuracy: 0.9852
Train Evaluation: [0.027746517211198807, 0.9902226328849792]
Test Evaluation: [0.046077828854322433, 0.9852371215820312]


In [8]:
y_fraud_pred = classifier.predict(X_fraud_test, batch_size=512, verbose=1)
y_fraud_pred = np.argmax(y_fraud_pred, axis=1)
print(accuracy_score(y_fraud_test, y_fraud_pred))

71/71 [==============================] - 0s 1ms/step
0.978007976955462
